In [1]:
import numpy as np
import tensorflow as tf
from core import *

In [2]:
IMG_DIM = 64
STRIP_WIDTH = 512
STRIP_HEIGHT = 512
SLICE = 1
SNAPS = 8 ##Number of snaps over 4 seconds
CHANNELS = 1
CLASSES = 2
# CHANNELS = 8

In [3]:
experiment = '8snap'
path = './datasets/' + experiment + '/'

_04847_img = np.load(path + '4847_' + experiment + '-image.npy')
_04799_img = np.load(path + '4799_' + experiment + '-image.npy')
_04820_img = np.load(path + '4820_' + experiment + '-image.npy')
_05675_img = np.load(path + '5675_' + experiment + '-image.npy')
_05680_img = np.load(path + '5680_' + experiment + '-image.npy')
_05710_img = np.load(path + '5710_' + experiment + '-image.npy')

_04847_lbl = np.load(path + '4847_' + experiment + '-label-onehot.npy')
_04799_lbl = np.load(path + '4799_' + experiment + '-label-onehot.npy')
_04820_lbl = np.load(path + '4820_' + experiment + '-label-onehot.npy')
_05675_lbl = np.load(path + '5675_' + experiment + '-label-onehot.npy')
_05680_lbl = np.load(path + '5680_' + experiment + '-label-onehot.npy')
_05710_lbl = np.load(path + '5710_' + experiment + '-label-onehot.npy')

In [4]:
train_img = np.vstack((_04847_img[5:,], _04799_img[5:,], _04820_img[5:,], _05675_img[5:,], _05680_img[5:,]))
train_lbl = np.vstack((_04847_lbl[5:,], _04799_lbl[5:,], _04820_lbl[5:,], _05675_lbl[5:,], _05680_lbl[5:,]))
val_img = np.vstack((_04847_img[:5,], _04799_img[:5,], _04820_img[:5,], _05675_img[:5,], _05680_img[:5,]))
val_lbl = np.vstack((_04847_lbl[:5,], _04799_lbl[:5,], _04820_lbl[:5,], _05675_lbl[:5,], _05680_lbl[:5,]))
#val_img = _05710_img
#val_lbl = _05710_lbl

In [5]:
print 'train_img', train_img.shape
print 'train_lbl', train_lbl.shape
print 'val_img', val_img.shape
print 'val_lbl', val_lbl.shape

train_img (375, 512, 512)
train_lbl (375, 2)
val_img (25, 512, 512)
val_lbl (25, 2)


In [6]:
x = tf.placeholder(tf.float32, shape=[None, STRIP_HEIGHT, STRIP_WIDTH], name='x')
x_image = tf.reshape(x, [-1, STRIP_HEIGHT, STRIP_WIDTH, CHANNELS], name='x_reshaped')
y_ = tf.placeholder(tf.float32, shape=[None, CLASSES], name='labels')
phase_train = tf.placeholder(tf.bool, name='phase_train')
keep_prob = tf.placeholder(tf.float32, name='keep_prob_fc1')

# 3 convolutional layers
conv1 = conv_layer(x_image, CHANNELS, 8, phase_train, 'conv1', 3, 3)
conv2 = conv_layer(conv1, 8, 16, phase_train, 'conv2', 5, 5)
conv3 = conv_layer(conv2, 16, 32, phase_train, 'conv3', 10, 10)
# maxpool

# 2 convolutional layers

# maxpool
conv3_flat = tf.reshape(conv3, [-1, 64 * 64 * 32]) # flatten conv2



fc1 = fc_layer(conv3_flat, 64 * 64 * 32, 1024, 'fc1') # fc_layer fc1
fc1_drop = tf.nn.dropout(tf.nn.relu(fc1), keep_prob) # dropout on fc1
fc2 = fc_layer(tf.nn.relu(fc1_drop), 1024, 512, name='fc2') # fc_layer fc2
y_conv = fc_layer(fc2, 512, 2, 'y_conv')    

In [7]:
max_epochs = 200
max_steps = 1000
learning_rate = 1e-5
step, train_acc_arr, train_loss_arr, val_acc_arr, val_loss_arr = 0, [], [], [], []
with tf.name_scope('xent'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    cross_entropy_count = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_count = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
# tf.summary.image('input', x_image, 3)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [8]:
sess = tf.InteractiveSession()
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter('/tmp/brain/4')
writer.add_graph(sess.graph)

sess.run(tf.global_variables_initializer())

for i in range(max_epochs):
    for batch_xs, batch_ys in next_batch(train_img, train_lbl, 5):
        feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5, phase_train: True}
#         if step % 5 == 0:
#             s = sess.run(merged_summary, feed_dict=feed_dict)
#             writer.add_summary(s, step)

        train_step.run(feed_dict=feed_dict)
        step += 1

        if step % 100 == 0:
            train_loss, train_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, train_img, train_lbl, 5)
            val_loss, val_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, val_img, val_lbl, 5)
            
            train_acc_arr.append(train_accuracy), train_loss_arr.append(train_loss)
            val_acc_arr.append(val_accuracy), val_loss_arr.append(val_loss)

            print "step %d, train accuracy %.4f, train loss %.5f, val accuracy %g, val loss %g" % (
                step, train_accuracy, train_loss, val_accuracy, val_loss)

step 100, train accuracy 0.5120, train loss 15.56715, val accuracy 0.52, val loss 15.1902
step 200, train accuracy 0.5973, train loss 8.30812, val accuracy 0.52, val loss 14.0535
step 300, train accuracy 0.4960, train loss 37.68283, val accuracy 0.56, val loss 22.501
step 400, train accuracy 0.6320, train loss 10.33414, val accuracy 0.64, val loss 20.7478
step 500, train accuracy 0.5920, train loss 12.34946, val accuracy 0.48, val loss 27.159
step 600, train accuracy 0.5893, train loss 15.18978, val accuracy 0.44, val loss 18.3089
step 700, train accuracy 0.6613, train loss 10.43840, val accuracy 0.56, val loss 26.3931
step 800, train accuracy 0.7360, train loss 4.16283, val accuracy 0.76, val loss 11.0633
step 900, train accuracy 0.7227, train loss 4.78370, val accuracy 0.64, val loss 11.7037
step 1000, train accuracy 0.7120, train loss 5.71632, val accuracy 0.64, val loss 13.9681


KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train,
                                         val_img, val_lbl, 100)
print 'Val Accuracy: %g, Val Loss: %g' % (test_accuracy * 100, test_loss)

In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
get_available_devices()

In [ ]:
val_img.shape

In [ ]:
val_img[1,:,:,1].shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
for i in range(8):
    plt.show(plt.imshow(val_img[1,:,:,i]))

In [ ]:
plt.show(plt.imshow(np.concatenate(val_img[1], axis=1)))

In [ ]:
plt.show(plt.imshow(np.concatenate((val_img[1,:,:,0], val_img[1,:,:,1], val_img[1,:,:,2], val_img[1,:,:,3], val_img[1,:,:,4], val_img[1,:,:,5], val_img[1,:,:,6], val_img[1,:,:,7]), axis=1)))

In [ ]:
plt.show(plt.imshow(np.concatenate(val_img[1,:,:,[0,1,2,3,4,5,6,7]], axis=1)))

In [ ]:
val_img[1].T.T.T.shape